# FAISS DB
- Facebook AI Similarity Search

In [ ]:
# !pip install faiss-gpu
# # !pip install faiss-cpu

### 영화 추천 시스템

In [1]:
import faiss
import numpy as np
import pandas as pd
import json
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
with open('data/movie_list_100.json', 'r', encoding='utf-8') as f:
    movies = json.load(f)

In [3]:
descriptions = [movie['description'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype=np.float32)

dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)    # table과 비슷한 데이터 구조 -> 효율적인 벡터 검색 가능 | IndexFlatL2: 거리(L2)기반 | (dim): 임베딩 차원 만큼의 형태
idx.add(desc_embed)

In [4]:
query_text = "복수극"
query_embed = np.array([model.encode(query_text)], dtype='float32')

top_n = 5
distances, indices = idx.search(query_embed, top_n)

print(distances)
print(indices)

[[0.6703637  0.67116827 0.75558627 0.7639918  0.7720646 ]]
[[33 11 29 36 50]]


In [5]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_id]['title']} (유사도 거리: {distances[0][i]:.2f})")

1번째 추천: 코코 (유사도 거리: 0.67)
2번째 추천: 반지의 제왕: 반지 원정대 (유사도 거리: 0.67)
3번째 추천: 하울의 움직이는 성 (유사도 거리: 0.76)
4번째 추천: 토이 스토리 (유사도 거리: 0.76)
5번째 추천: 007 노 타임 투 다이 (유사도 거리: 0.77)


### 사용자 맞춤 뉴스 추천 <실습>

In [6]:
# 뉴스의 제목을 10, 20 크롤링
# 키워드 입력
# faiss 활용해 맞춤 뉴스 추천